# PyConEs Canarias 2023

## Utilizando LLMs como nuevo paradigma de programacion

### Jacinto Arias - Taidy Cloud
#### jacinto.arias@taidy.cloud

---

En esta libreta vamos a poner en práctica los conceptos de LLMs vistos durante el taller, utilizaremos diversas librerías, como openai, langchain o gptall.

- Puedes ejecutar toda la libreta en un pc convencional, simplemente instala los paquetes usando tu herramienta favorita. Las celdas de GPT4All (`llm_local`) es muy probable que no funcionen en equipos windows sin configuración adicional
- Algunas celdas no se pueden ejecutar porque requieren servicios que no cubrimos en el taller, estarán marcadas como __DEMO!__
- Algunas celdas descargan datos en la cache de tu máquina, acuérdate de limpiarla cuando termines.

# Paquetes básicos y configuración

Iremos importando la funcionalidad conforme la necesitemos.

## Dotenv

Usaremos python-dotenv para cargar nuestra clave de OPEN_AI como variable de entorno, deberás crear el fichero .env correspondiente en tu entorno y pegar ahi la clave de OpenAI

__Recuerda no compartir esta clave ni subir este fichero a un repo!__

In [ ]:
import dotenv
import os
import json

In [ ]:
dotenv.load_dotenv()

# Demo 1: Usando OpenAI desde python

En esta demo cubriremos el uso básico de OpenAI y su API de `completions` para trabajar con los modelos GPT desde python

In [ ]:
import openai

## Completion (Legacy)

La API de completions nos permite generar texto de manera predictiva en función de un input

In [ ]:
completion = openai.Completion.create(
    model="text-davinci-003", 
    prompt="The recipe for carbonara is",
    temperature=1,
    max_tokens=20,
)

In [ ]:
completion

In [ ]:
print(completion.choices[0].text)

# DEMO 02: Prompt Engineering 101

En esta demo veremos varias técnicas de prompting básico

- Zero Shot
- Few Shot
- Modelos entrenados como Chat
- Otros modelos (GPT4All + Anthropic via AWS Bedrock)

## Few Shot Prompt

Un prompt directo que confía únicamente en la capacidad de los parámetros de la red

In [ ]:
chef_prompt = """
You are a helpful assistant with expert chef knowledge. 
You will provide instructions to the user on how to make a recipe by mentioning first ingredients, 
measures as well as instructions.

User: Give me the recipe for spanish potato omelette
Assistant:
"""

In [ ]:
completion = openai.Completion.create(
    model="davinci-002", 
    prompt=chef_prompt,
    temperature=0.4,
    max_tokens=100,
)

In [ ]:
print(completion.choices[0].text)

### Zero shot mejorado

Siempre podemos refinar nuestro prompt acotando la tarea a realizar, por ejemplo con instrucciones sobre cómo queremos que formatee la salida.

In [ ]:
chef_prompt = """
You are a helpful assistant with expert chef knowledge. You will provide instructions to the user on how to make a recipe.

In your recipes you will provide information using bullet points about:

- Servings: Number of servings you are measuring for
- Ingredients: List of ingredients and measures
- Instructions: Detailled instructions, step by step


User: Give me the recipe for spanish potato omelette
Assistant:
"""

In [ ]:
completion = openai.Completion.create(
    model="davinci-002", 
    prompt=chef_prompt,
    temperature=0.4,
    max_tokens=100,
)

In [ ]:
print(completion.choices[0].text)

## Few Shot Prompts

Los LLMs no son muy potentes a la hora de generalizar problemas y suelen pecar de dispersarse y no resolver los problemas correctos.

Proporcionar ejemplos en nuestro prompt puede ayudarnos a mejorar la calidad de las respuestas. Fíjate que el propio ejemplo puede servir para condicionar el formato de la salida.

In [ ]:
chef_prompt = """
You are a helpful assistant with expert chef knowledge. You will provide instructions to the user on how to make a recipe.

In your recipes you will provide information about:

- Number of servings you are measuring for
- List of ingredients and measures
- Detailled instructions, step by step

User: Give me the recipe for valencian paella
Assistant: *Servings:* 6 people

*Ingredients:*
- 1/2 cup olive oil
- 500g chicken, cut into pieces
- 500g rabbit, cut into pieces
- 200g green beans
- 100g butter Beans
- saffron and paprika season
- A tbsp of fresh grated tomato
- salt and pepper
- 400g rice
- 3 parts of water per rice

*Steps:*
- Heat the paella pan with the olive oil
- Season and fry the chicken and rabbit until golden brown (40m)
- Add the green beans and butter beans
- Add the tomato saffron and paprika and cook for 5m
- Add the water and bring to boil for 20m
- Add the rice and cook for another 20m

User: Give me the recipe for spanish potato omelette
Assistant:
"""

In [ ]:
completion = openai.Completion.create(
    model="davinci-002", 
    prompt=chef_prompt,
    temperature=0.4,
    max_tokens=150,
)

In [ ]:
print(completion.choices[0].text)

## Chat Completions

Los modelos de chat están específicamente entrenados (ver RLHF) para poder responder mejor a las instrucciones en los prompts. Para ello, simulan una conversación que permite modelar mejor la interacción y la ejecución de acciones, pero en el fondo es muy similar a lo que hacen el resto de modelos.

En OpenAI son los modelos principales como GPT-3.5 y GPT-4

In [ ]:
messages = [
    {
    "role": "system", 
    "content": """
        You are a helpful assistant that help people with recipes and dishes
    """
    },
    {
    "role": "user", 
    "content": "What is the recipe for carbonara?"
    }
]

In [ ]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    messages=messages
)

In [ ]:
completion

### System Messages

El system message es un mensaje muy especial que puede utilizarse para proporcionar instrucciones, pero hay que tener cuidado, muchos modelos no hacen caso de el...

Como podéis ver en el caso de GPT-3.5 puede condicionar totalmente el resultado.

__NOTA 👹:__ Usar un LLM para potenciar estereotipos y sesgos está __muy mal__ esta es solo una demo para que veais lo bien que se le da...

In [ ]:
messages = [
    {
        "role": "system", 
        "content": """
            You are a helpful assistant that help people with recipes and dishes, 
            you are an expert in italian cuisine and value and respect tradition, 
            you will describe the recipes evocating your memories about your past and your nona, 
            simulate an italian accent when typing.
        """
    },
    {
        "role": "user", 
        "content": "What is the recipe for carbonara?"
    }
]

In [ ]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    messages=messages
)

In [ ]:
print(completion["choices"][0]["message"].content)

### Flujo de conversación + Few Shot Learning

Productos como ChatGPT están implementados para mejorar los flujos de conversación de los modelos y proporcionar esa experiencia de usuario tan conocida. En la API nos toca implementarlo a nosotros concatenando los mensajes...

In [ ]:
messages.append(dict(completion["choices"][0]["message"]))
messages.append({
    "role": "user",
    "content": "So shall I put lots of cream in it?"
})

In [ ]:
messages

In [ ]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    messages=messages
)

In [ ]:
print(completion["choices"][0]["message"].content)

### De vuelta a Few Shot Prompting

El flujo conversacional se puede utilizar para enfatizar el comportamiento del Few Shot Learning

In [ ]:
messages = [
    {
        "role": "system", 
        "content": """
            You are a helpful assistant with expert chef knowledge. 
            You will provide instructions to the user on how to make a recipe.
        """
    },
    {
        "role": "user", 
        "content": "What is the recipe for valencian paella?"
    },
    {
        "role": "system",
        "content": """*Servings:* 6 people

*Ingredients:*
- 1/2 cup olive oil
- 500g chicken, cut into pieces
- 500g rabbit, cut into pieces
- 200g green beans
- 100g butter Beans
- saffron and paprika season
- A tbsp of fresh grated tomato
- salt and pepper
- 400g rice
- 3 parts of water per rice

*Steps:*
- Heat the paella pan with the olive oil
- Season and fry the chicken and rabbit until golden brown (40m)
- Add the green beans and butter beans
- Add the tomato saffron and paprika and cook for 5m
- Add the water and bring to boil for 20m
- Add the rice and cook for another 20m
        """
    },
    {
        "role": "user", 
        "content": "What is the recipe for spanish potato omelette?"
    }
]

In [ ]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    messages=messages
)

In [ ]:
print(completion["choices"][0]["message"].content)

### Few Shot + Ground Truth

Si lo modelamos bien, la técnica de Few Shot Prompting no deja de ser una especie de aprendizaje supervisado.

En la teoría hay aproximaciones que consideran estos ejemplos como parte de los parámetros de la red e incorporan técnicas de aprendizaje y optimización al prompt (ver anexo a la charla).

In [ ]:
prompt_messages = [
    {
        "role": "system", 
        "content": """
            You are an text classification tool that classifies sentences given the content for a telco company. 
            The sentences come from customers calls to an IVR stating the intent of their call,
            and must be routed to he corresponding department, 
            the available labels are: "sales", "support", "billing", "other"
        """
    },
    {
        "role": "user", 
        "content": "Buenos días, he recibido un cargo por duplicado de mi ultima factura y me gustaría recibir un reembolso"
    },
    {
        "role": "system",
        "content": "billing"
    },
    {
        "role": "user", 
        "content": "Mi linea movil no funciona y es la tercera vez que llamo, estoy desesperada, necesito que me atiendan ya"
    },
    {
        "role": "system",
        "content": "support"
    },
    {
        "role": "user", 
        "content": "Me gustaría contratar una linea movil adicional"
    },
    {
        "role": "system",
        "content": "sales"
    },
    {
        "role": "user", 
        "content": "¿Hola? Uy vaya me he equivocado.. Pepe!! ¿Pero qué numero me has dado?"
    },
    {
        "role": "system",
        "content": "other"
    },
]

In [ ]:
def classify_message(msg):
    prompt = prompt_messages + [
        {
            "role": "user", 
            "content": msg
        }
    ]

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", 
        messages=prompt
    )

    return completion["choices"][0]["message"].content

In [ ]:
classify_message("Hola? No encuentro la manera de descargar mi ultima factura de vuestra web")

In [ ]:
classify_message("Mi gato ha tirado el router al suelo y se ha roto")

In [ ]:
prommt_messages = [
    {
        "role": "system", 
        "content": """
            You are an text classification tool that classifies sentences given the content for a telco company. 
            The sentences come from customers calls to an IVR stating the intent of their call and must be routed 
            to he corresponding department, the available labels are: "sales", "support", "billing", "other" 
            as well as by the sentiment of the message: "positive", "negative", "neutral"
        """
    },
    {
        "role": "user", 
        "content": "Buenos días, he recibido un cargo por duplicado de mi ultima factura y me gustaría recibir un reembolso"
    },
    {
        "role": "system",
        "content": """{"department": "billing", "sentiment": "neutral"}"""
    },
    {
        "role": "user", 
        "content": "Mi linea movil no funciona y es la tercera vez que llamo, estoy desesperada, necesito que me atiendan ya"
    },
    {
        "role": "system",
        "content": """{"department": "support", "sentiment": "negative"}"""
    },
    {
        "role": "user", 
        "content": "Me gustaría contratar una linea movil adicional"
    },
    {
        "role": "system",
        "content": """{"department": "sales", "sentiment": "neutral"}"""
    },
    {
        "role": "user", 
        "content": "¿Hola? Uy vaya me he equivocado.. Pepe!! ¿Pero qué numero me has dado?"
    },
    {
        "role": "system",
        "content": """{"department": "billing", "sentiment": "negative"}"""
    },
]

In [ ]:
classify_message("Hola? No encuentro la manera de descargar mi ultima factura de vuestra web, que mal funciona")

In [ ]:
classify_message("Mi gato ha tirado el router al suelo y se ha roto!!, porfi majetes podéis enviarme un técnico?, lo necesito con urgencia! muchas gracias!!")

## Hay vida fuera de OpenAI

En la práctica todas estas técnicas deberían funcionar de manera genérica con cualquier LLM, sin embargo el comportamiento de estos modelos puede variar en función de cada modelo por su arquitectura, complejidad... fine tuning aplicado...

### Hugging Face 🤗 y modelos open

Sin duda el mejor lugar para explorar, simplemente ten en cuenta temas de licencia y código ético.

Para muchos modelos como Falcon, necesitarás una infraestructura costosa (puedes usar un hyperscaler), otras alternativas te permitirán depurar e incluso implantar la funcionalidad de un LLM en tu propio laptop, como es el caso e las redes cuantizadas como GPT4All o llama.cpp

#### Este codigo necesita una GPU potente no ejecutar!!!

In [ ]:
# import transformers
# import torch
# from transformers import AutoTokenizer, pipeline

# model = "tiiuae/falcon-40b-instruct"

# tokenizer = AutoTokenizer.from_pretrained(model)

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
#     device_map="auto",
# )

In [ ]:
# sequences = pipeline(
#    "The ingredients of carbonara are:",
#     max_length=50,
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
# )

# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")

### GPT4All

__ATENCION__ Este código no funcionará correctamente en un entorno windows sin configurar... ver el repo de GTP4All o usa una alternativa como google collab

https://github.com/nomic-ai/gpt4all/tree/main/gpt4all-bindings/python

In [ ]:
from gpt4all import GPT4All

In [ ]:
model = GPT4All(
    model_name='ggml-model-gpt4all-falcon-q4_0.bin'
)

In [ ]:
output = model.generate(
    """
You are a helpful assistant with expert chef knowledge. You will provide instructions to the user on how to make a recipe.

User: Give me the recipe for valencian paella
Assistant: *Servings:* 6 people

*Ingredients:*
- 1/2 cup olive oil
- 500g chicken, cut into pieces
- 500g rabbit, cut into pieces
- 200g green beans
- 100g butter Beans
- saffron and paprika season
- A tbsp of fresh grated tomato
- salt and pepper
- 400g rice
- 3 parts of water per rice

*Steps:*
- Heat the paella pan with the olive oil
- Season and fry the chicken and rabbit until golden brown (40m)
- Add the green beans and butter beans
- Add the tomato saffron and paprika and cook for 5m
- Add the water and bring to boil for 20m
- Add the rice and cook for another 20m

User: Give me the recipe for spanish potato omelette
Assistant:
    """, 
    max_tokens=300,
    # temp=0.6,
    # top_k=10,
)

In [ ]:
print(output)

# Langchain 🦜⛓️

Langchain es una librería que nos a a permitir abstraer el uso de LLMs y de las técnicas de prompting y gestión de conocimiento más avanzadas.

Su documentación es un poco caótica y debe utilizarse con cautela, pues es un proyecto muy vivo y con mucho enfoque de investigación, pero es perfecto para aprender y prototipar, en el futuro quizás sea una buena opción para productivizar este tipo de software.

## Langchain + OpenAI

La mayoría de ejemplos que os encontraréis ahi fuera estarán basados en el uso de modelos de OpenAI con langchain, si habéis cargado antes vuestra API Key, este código debería funcionaros

El módulo más básico de langchain es el de LLMs, y permite abstraer los modelos de una manera sencilla

In [ ]:
import langchain
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(model_name="text-davinci-003")

In [ ]:
print(llm("The recipe for carbonara is"))

## Prompt debugging

Una de las cosas buenas de langchain es que implementan una gran cantidad de utilidades en cada modelo y componente que integran, como la capacidad de monitorizar el consumo de la API

In [ ]:
result = llm.generate(["The recipe for carbonara is"])

In [ ]:
result.llm_output

In [ ]:
result.generations

## Langchain Prompts

El segundo módulo más importante de Langchain es la capacidad de gestionar prompts, algunos critican que es solo una manera compleja de utilizar _string templates_ pero para otros es una aproximación a un futuro paradigma de modelado y composición de funcionalidad, como la orientación a objetos.

Langchain nos permite definir plantillas de prompts para reutilizarlos y abstraer el contenido de los mismos, convirtiéndolos en funciones de input

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt_template = PromptTemplate.from_template(
    "Tell me the recipe for {dish}"
)

prompt_template.format(dish="carbonara")

In [ ]:
prompt = prompt_template.format(dish="carbonara")

## Langchain + Ecosistema LLM

Como hemos comentado, langchain abstrae el uso de diversas herramientas y modelos, esto nos permite cargar diversos modelos dentro de la plataforma de manera transparente

In [ ]:
from langchain.llms import GPT4All

In [ ]:
llm_local = GPT4All(model="ggml-model-gpt4all-falcon-q4_0.bin")


In [ ]:
print(llm_local.predict(prompt))

## AWS Bedrock

__SOLO DEMO__ para utilizar esta celda se requiere una cuenta de AWS configurada para poder utilizar bedrock

Este es un servicio de AWS que permite acceder a modelos de otras compañías en un formato de pago por uso, AWS ha decidido trabajar directamente en una integración con langchain para poderlo poner en marcha.

Una vez cargado el LLM, el uso de la API es completamente abstracto

In [ ]:
from langchain.llms import Bedrock

llm_aws = Bedrock(
    credentials_profile_name="courses",
    region_name="us-east-1",
    model_id="anthropic.claude-instant-v1",
)

In [ ]:
print(llm_aws.predict(prompt))

## Few Shot Template

La apuesta principal de langchain es proporcionar abstracciones para cualquier tipo de técnica de prompt engineering, como por ejemplo el uso del few shot.

Esto permite más adelante integrar el few shot con otro tipo de técnicas más complejas como la selección de ejemplos.

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate

In [ ]:
examples = [
    {
        "question": "Buenos días, he recibido un cargo por duplicado de mi ultima factura y me gustaría recibir un reembolso",
        "answer": """{{"department": "billing", "sentiment": "neutral"}}"""
    },
    {
        "question": "Mi linea movil no funciona y es la tercera vez que llamo, estoy desesperada, necesito que me atiendan ya",
        "answer": """{{"department": "support", "sentiment": "negative"}}"""
    },
    {
        "question": "Me gustaría contratar una linea movil adicional",
        "answer": """{{"department": "sales", "sentiment": "neutral"}}"""
    },
    {
        "question": "¿Hola? Uy vaya me he equivocado.. Pepe!! ¿Pero qué numero me has dado?",
        "answer": """{{"department": "billing", "sentiment": "negative"}}"""
    }
]

In [ ]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Question: {input}\nAnswer:", 
    input_variables=["input"]
)

print(prompt.format(input="Hola? No encuentro la manera de descargar mi ultima factura de vuestra web, que mal funciona"))

In [ ]:
result = llm.predict(prompt.format(input="Hola? No encuentro la manera de descargar mi ultima factura de vuestra web, que mal funciona"))

In [ ]:
result

## Langchain Chat Models

La abstracción de prompts también nos va a permitir modelar aspectos como las conversaciones con construcciones abstractas, muy importante pues no todos los modelos de Chat se comportan igual y previsiblemente en el futuro veremos todo tipo de APIs y modelos en el mercado...

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
chat_llm = ChatOpenAI()

In [ ]:
result = chat_llm.generate([[
    SystemMessage(content="You are a helpful assistant that help people with recipes and dishes"),
    HumanMessage(content="What is the recipe for carbonara?")
]])

print(result.generations[0][0].text)

In [ ]:
result.dict().keys()

In [ ]:
result.llm_output

### Model and prompt abstraction

In [ ]:
prompt_messages = [
    SystemMessage(content="""
        You are an text classification tool that classifies sentences given the content for a telco company. 
        The sentences come from customers calls to an IVR stating the intent of their call,
        and must be routed to he corresponding department, 
        the available labels are: "sales", "support", "billing", "other"
    """),
    HumanMessage(content="Buenos días, he recibido un cargo por duplicado de mi ultima factura y me gustaría recibir un reembolso"),
    AIMessage(content="billing"),
    HumanMessage(content="Mi linea movil no funciona y es la tercera vez que llamo, estoy desesperada, necesito que me atiendan ya"),
    AIMessage(content="support"),
    HumanMessage(content="Me gustaría contratar una linea movil adicional"),
    AIMessage(content="sales"),
    HumanMessage(content="¿Hola? Uy vaya me he equivocado.. Pepe!! ¿Pero qué numero me has dado?"),
    AIMessage(content="other"),
]

In [ ]:
from langchain.prompts.chat import HumanMessagePromptTemplate

In [ ]:
input_prompt = HumanMessagePromptTemplate.from_template(input_variables=["question"], template="{question}")

In [ ]:
prompt = prompt_messages + input_prompt.format_messages(question="Hola? No encuentro la manera de descargar mi ultima factura de vuestra web")

In [ ]:
result = chat_llm.generate([prompt])

print(result.generations[0][0].text)

## Langchain Chains

El potencial de un LLM se multiplica exponencialmente cuando somos capaces de combinarlo con herramientas o de construir comportamientos complejos encadenando diversos prompts para que completen una tarea más compleja entre todos, simplificando cada parte y focalizando el esfuerzo en cada prompt.

Probablemente el tercer módulo más importante de langchain es la capacidad de modelar el uso de cadenas

Vamos a construir una cadena básica con un solo LLM

In [ ]:
from langchain.chains import LLMChain

In [ ]:
prompt_template = "What are the ingredients for {dish}"

llm = OpenAI(temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

print(llm_chain.predict(dish="Valencian Paella"))

### Output Parsers

Vamos a incorporar una herramienta básica, que no es un LLM, sino una manera de componer prompts y añadir una funcionalidad básica, en este caso, devolver una salida procesada en lugar de texto en bruto.

Re-implementar este comportamiento con tu propio código es muy sencillo, langchain apuesta por la estandarización y la capacidad de integración gracias a su Hub y sus integraciones

- https://python.langchain.com/docs/integrations/providers
- https://smith.langchain.com/hub


In [ ]:
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain


In [ ]:
prompt_template = """
List all the ingredients for {dish}
{parser_instructions}
"""

output_parser = CommaSeparatedListOutputParser()
prompt = PromptTemplate(
    template=prompt_template, 
    input_variables=["dish"], 
    partial_variables={"parser_instructions": output_parser.get_format_instructions()},
)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    output_parser=output_parser
)


In [ ]:
handler = StdOutCallbackHandler()

In [ ]:
llm_chain(
    "Valencian Paella", 
    # callbacks=[handler]
)

### Más parsers

Langchain proporciona otros parsers o simplemente ideas para integrar tus LLMs, por ejemplo, facilitando la salida estructurada de los resultados

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
from enum import Enum


In [ ]:
class Invoice(BaseModel):
    datetime: str = Field()
    provider_name: str = Field()
    provider_vat: str = Field()
    num_items: int = Field()
    subtotal: float = Field()
    tax: float = Field()
    total: float = Field()

In [ ]:
parser = PydanticOutputParser(pydantic_object=Invoice)

prompt = PromptTemplate(
    template="""
    You are a tool that parses information from the raw text generated by scanning a physical ticket using an OCR

    Extract the information following the instructions below:
        datetime: Date and time of the transaction
        provider_name: Date of the provider
        provider_vat: VAT number of the provider, eg: ES12345678A
        num_items: Number of items in the ticket
        subtotal: Subtotal of the ticket
        tax: Tax of the ticket
        total: Total of the ticket


    {format_instructions}

    OCR Text: {text}

    Result:
    """,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    output_parser=parser
)



In [ ]:
results = llm_chain.apply(
    [
        {
            "text": """
            Factura Simplificada no 345456w/22
            12/10/2023 12:34:56
            Restaurante Benito

            Calle Mayor 25, Albacete
            54726418B

            Articulos
            Coca Cola Zero  1 2.00€ 2.00€
            Caña Pequeña 1 1.25€ 1.25€
            Marineras 2 3.00€ 6.00€
            Subtotal 9.25€
            IVA 21% 1.94€
            Total 11.19€
            """
         },
        {
            "text": """
            Gasolineras de la mancha
            Factura Simplificada
            Diesel A+ 0.65€/L 80L 52.00€

            Iva 21 incluido

            Matricula 1234ABC
            Autovía de Alicante Km134, Almansa
            Gasoil Mancha SL 54726418B
            """
        }
    ]
)

In [ ]:
results[0]["text"].model_dump()

In [ ]:
results[1]["text"].model_dump()

### Sequential Chain

La funcionalidad más potente de utilizar cadenas es la capacidad de encadenar varios LLMs para resolver una tarea compleja, especializando cada uno de los prompts.

El siguiente ejemplo muestra la capacidad de modelado básico de un sistema que resume un chat corporativo con capacidad de filtrado de contenido.

In [ ]:
summary_template = """
    You are a helpful assistant that will summary conversations among employees that use a company chat in a public channel using one sentence and in spanish

    Original Text: {conversation}
    Summary:
"""

summary_prompt_template = PromptTemplate(input_variables=["conversation"], template=summary_template)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt_template, output_key="summary")

In [ ]:
summary_chain("""
    Juan: Me ha llamado el responsable de la empresa Compuglobalhypermeganet para que les hagamos un nuevo desarrollo sobre lo que ya hicimos el año pasado
    Marta: Hola Juan, te refieres al proyecto de instalacion de los nuevos servidores o la migración de su sistema de facturación
    Juan: La migración de su sistema de facturación, quiren un nuevo módulo para gestionar una nueva pasarela de pago online
    Marta: Ok, pues me pongo con ello, les envío un presupuesto y les pido una reunión para aclarar los detalles, @Sara te asigno el proyecto a tu cuenta para que hagas seguimiento de la oferta
    Sara: Ok, gracias Marta
""")

In [ ]:
summary_chain("""
    Pedro: @Antonio figura!! menuda llevabas el sabado, no se ni como has venido a currar hoy
    Antonio: Calla tu, menuda cogorza, después de la **%&$ de semana que me dio el pesado este solo queria olvidarme
    Pedro: Este jefe cada dia más tonto
    Alba: Para tontos vosotros que este es el canal de cotizaciones y lo esta viendo todo el mundo 😅
    Pedro: Como puedo borrar los mensajes?
""")

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
moderation_template = """
    You are a moderation and filtering tool that will review texts containing the summary of a company conversation
    If Original Text is related to a business topic you will return the same text, untouched, otherwise you will return OFFTOPIC

    Original Text: {summary}
    Filtered Text:
"""

moderation_prompt_template = PromptTemplate(input_variables=["summary"], template=moderation_template)
moderation_chain = LLMChain(llm=llm, prompt=moderation_prompt_template, output_key="filtered_summary")

In [ ]:
filtered_summary_chain = SequentialChain(
    chains=[summary_chain, moderation_chain],
    input_variables=["conversation"],
    output_variables=["filtered_summary"],
)

In [ ]:
filtered_summary_chain("""
    Juan: Me ha llamado el responsable de la empresa Compuglobalhypermeganet para que les hagamos un nuevo desarrollo sobre lo que ya hicimos el año pasado
    Marta: Hola Juan, te refieres al proyecto de instalacion de los nuevos servidores o la migración de su sistema de facturación
    Juan: La migración de su sistema de facturación, quiren un nuevo módulo para gestionar una nueva pasarela de pago online
    Marta: Ok, pues me pongo con ello, les envío un presupuesto y les pido una reunión para aclarar los detalles, @Sara te asigno el proyecto a tu cuenta para que hagas seguimiento de la oferta
    Sara: Ok, gracias Marta
""")

In [ ]:
filtered_summary_chain("""
    Pedro: @Antonio figura!! menuda llevabas el sabado, no se ni como has venido a currar hoy
    Antonio: Calla tu, menuda cogorza, después de la **%&$ de semana que me dio el pesado este solo queria olvidarme
    Pedro: Este jefe cada dia más tonto
    Alba: Para tontos vosotros que este es el canal de cotizaciones y lo esta viendo todo el mundo 😅
    Pedro: Como puedo borrar los mensajes?
""")

# Gestión del conocimiento y la memoria

Sin duda uno de los básicos que debes añadir a tus aplicaciones que usen LLMs es la gestión de datos propios que no hayan sido utilizados para su entrenamiento. Este es el componente clave que te permitirá construir aplicaciones que resuelvan problemas únicos y novedosos.

Para ello langchain proporciona una serie de utilidades que nos permitirán incorporar tecnologías como las bases de datos de vectores

## Datos y conocimiento

Recuerda que los LLMs están entrenados en un momento del pasado, con datos finitos y que por tanto su conocimiento de la realidad es limitado.

El problema es que muchos modelos tienden a inventarse (alucinar) las respuestas, y esto es algo muy peligroso

In [ ]:
result = chat_llm.generate([[
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Que es la pycones, dónde y cuándo se celebra?")
]])

print(result.generations[0][0].text)

In [ ]:
print(llm.predict("Que es la pycones, dónde y cuándo se celebra?"))

In [ ]:
print(llm_local.predict("What is the pycones, when and where is it held?"))

## Prompt Stuffing

La técnica más sencilla para resolver este problema es añadir a nuestro prompt el conocimiento que pueda faltarle al LLM.

Esto puede ser muy sencillo en el caso de tareas básicas de resumen de textos, pero para aplicaciones ambiciosas y genéricas puede ser insuficiente.

Además incrementar notablemente el tamaño de nuestros prompts tiene clara desventajas y limitaciones:

- Estamos limitados al número de tokens de la ventana contextual del modelo
- Incrementamos el precio
- Los modelos tienen a divagar y no focalizarse en los contenidos de prompts largos y vagos


El el siguiente ejemplo utilizaremos técnicas básicas de extracción de datos de web para aportarle contexto a nuestro modelo.

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
source_urls = [
    "https://2023.es.pycon.org/",
    # "https://2023.es.pycon.org/faq/",
    # "https://2023.es.pycon.org/ciudad/",
    # "https://2023.es.pycon.org/viaje/",
    # "https://2023.es.pycon.org/patrocinios/",
]

In [ ]:
context = [
    t.get_text()
    for url in source_urls
    for t in BeautifulSoup(requests.get(url).text).find_all('p')
]

In [ ]:
context[0]

In [ ]:
context_concat = " ".join(context)

In [ ]:
len(context_concat)

In [ ]:
prompt_template = PromptTemplate.from_template("""
Answer the following questions using the following contextual information, 
if you do not know the answer respond clearly "I Do not know"\n 

Context: {context}
                                               
Question: {question}
                                               
Answer:
""")


In [ ]:
llm.predict(prompt_template.format(
    context=context_concat, 
    question="Que es la PyConEs y donde se celebra"
))

## Retrieval Augmented Generation & Vector Stores

La manera sofisticada de resolver el problema anterior es utilizar técnicas de gestión de la información no estructurada.

Esto implica la incorporación de técnicas novedosas de recuperación de información (donde también intervienen los LLMs). 

Concretamente, langchain está especializado en el uso de bases de datos de vectores, como chroma, FAISS o pinecone, que aprovechan la potencia de los LLMs para trabajar con embeddings.

El funcionamiento básico consiste en transformar el prompt en una representación vectorial semántica (embedding) y realizar una búsqueda utilizando algún algoritmo como la similaridad. Esto nos abre muchas posibilidades para elegir tipos de software, como la base de datos concreta, el modelo de embedding, algoritmo de búsqueda así como las técnicas de transformación de prompts al buscar y recopilar la información.

En el ejemplo siguiente vamos a usar una base de datos local usando chroma para incluir como documentos los párrafos de las webs y asi limitar enormemente el tamaño del prompt generado.

Nos apoyaremos en langchain y su cadena predefinida de búsqueda en bases de datos contextuales.

In [ ]:
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
raw_documents = [Document(page_content=c) for c in context]
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [ ]:
docs = db.similarity_search("aeropuerto", k=5)
for d in docs:
    print(d)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler


In [ ]:
retriever = db.as_retriever()

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    verbose=True
)

In [ ]:
handler = StdOutCallbackHandler()


In [ ]:
qa(
    "cuantas charlas hay en la pycones?", 
    # callbacks=[handler]
)

## Langchain helpers

Langchain proporciona una cantidad ingente de integraciones y componentes proporcionados por la comunidad, que aceleran el prototipado y la incorporación de herramientas en todos los procesos de prompting, gracias a su naturaleza modular es cuestión de cada uno utilizar estos helpers o desarrollar la integración de manera independiente.

En el siguiente script haremos un "scrap" de todas las charlas de la pycones 2023 para tener un bot que nos permita responder a preguntas sobre el programa de la conferencia.

In [ ]:
from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import BeautifulSoupTransformer

from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain

In [ ]:
pycones_talks_urls = [  
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/9QSL79/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/YMZVVQ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/PBDTVD/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/KHELNK/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/KW33VH/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/SWP7AZ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/7UAM7P/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/FXYFQZ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/RDNEXC/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/78GAHC/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/U78RSY/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/YP9PL9/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/ZZHMWW/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/7YT33P/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/F98UXU/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/FZKJSN/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/Z9YG88/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/LZ8FWD/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/KQVXVV/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/ZQ778X/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/UQUJVP/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/MXLC8T/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/HCMMW7/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/PT3LWB/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/PVJES3/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/ARSG8Q/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/FPMUUQ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/YZ3TU3/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/8FYCPY/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/UVLLJE/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/RXM3KK/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/VZMNRK/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/X9H9U9/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/P3YLBP/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/C7FLWF/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/NYSCCZ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/9NYMMU/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/N9BWSG/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/QKZNTQ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/EQJCFN/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/3ZSEBF/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/XDTUDT/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/9F9WMA/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/TGYBY3/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/TMTRB9/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/VDLCXR/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/V9VX9M/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/EKEZVZ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/DYL7CA/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/7X3PPN/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/EDNDH7/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/KWRG7N/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/KCGKWT/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/GMWRLP/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/D88YTV/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/HBCMXE/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/XFWAZV/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/MXBJHM/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/BGQ8RJ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/XNQUSH/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/BGVWFX/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/UT33TX/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/HJGQLB/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/VFBZDS/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/HKVWAA/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/UTWCZ3/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/YLX3NS/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/YQ7RLM/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/DVCBZU/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/87Y7CW/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/ZYCPG3/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/YW79NH/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/SVLHVA/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/SCXJ3N/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/ZMHSVG/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/MMAXDN/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/XXAXQJ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/N9ACAB/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/7ZEHGA/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/SWFPXZ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/HPGZA8/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/JYFQBL/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/AYNPNM/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/UXHEWC/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/KFAXTU/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/ZPKG73/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/VSTZCL/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/7KDMK8/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/NCJBTE/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/CQ8EQD/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/R9K7KT/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/A3BRFE/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/QSLHJE/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/HBLX8F/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/7BWZGN/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/JRZ8LK/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/ZQSMMF/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/QUXVJQ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/JN7CTD/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/GRWUT3/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/ASD8DD/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/AGDKBR/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/7VCRGQ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/9NGPJT/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/FSRU8J/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/LABN9C/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/9K7AZQ/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/DCUGRW/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/Z9KMUT/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/7KGKEN/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/YWFFQM/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/7ZZZ7D/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/PJXQQM/',
    'https://charlas.2023.es.pycon.org/pycones-2023/talk/Q8C3EZ/'
]

In [ ]:
loader = AsyncHtmlLoader(pycones_talks_urls)

In [ ]:
docs = loader.load()

In [ ]:
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(docs, tags_to_extract=["p"])

In [ ]:
documents = text_splitter.split_documents(docs_transformed)
db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [ ]:
retriever = db.as_retriever(
    search_type="similarity",
    # search_type="mmr",
    search_kwargs={"k": 15}
)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="map_reduce", 
    retriever=retriever,
    verbose=True
)

In [ ]:
result = qa(
    "que charlas hablarán de web scraping? ", 
    callbacks=[handler]
)

In [ ]:
result["result"]

## Conversational memory

Una de las características más potentes de ChatGPT es su capacidad para recordar eventos pasados durante una conversación, si utilizamos la API hay que recrear este comportamiento a mano, pero por suerte langchain tiene distintos tipos de utilidades para gestionar la memoria durante la interacción con un LLM en una conversación.

In [ ]:
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory

In [ ]:
memory = ConversationBufferMemory()
# memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))

In [ ]:
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=chat_llm, 
    memory=memory,
    verbose=True, 
)

In [ ]:
conversation.predict(input="Hi there!")

In [ ]:
conversation.predict(input="I live in valencia and I would like to travel by car to madrid, how much time will the trip be?")

In [ ]:
conversation.predict(input="And if i want to stop in Albacete on the way?")

# Agents y Tools

Un poco experimental, pero Langchain proporciona la capacidad de crear comportamientos autónomos gracias a la combinación de prompts reflexivos y herramientas, este framework llamado ReAct permite generar comportamientos para resolver problemas de manera autónoma.

In [ ]:
from langchain.agents import AgentType, initialize_agent, Tool, load_tools
from langchain.memory import ConversationBufferMemory

from langchain.tools import DuckDuckGoSearchResults
from langchain import LLMMathChain


In [ ]:
search = DuckDuckGoSearchResults()

In [ ]:
search("how high is mount teide?")

In [ ]:
tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    )
] + load_tools(["llm-math"], llm=llm)


In [ ]:
agent_executor = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent_executor.invoke({"input": "how high is mount teide?"})


In [ ]:
agent_executor.invoke({"input": "how high is mount teide compared to everest?"})


In [ ]:
agent_executor.invoke({"input": "how high is mount teide compared to the tallest mountain in the world?"})
